In [1]:
# Step 1
import pandas as pd
from pathlib import Path
import numpy as np

In [2]:
# Files needed
school_data_path = Path("Resources/schools_complete.csv")
student_data_path = Path("Resources/students_complete.csv")

In [3]:
# Read store into Pandas DataFrames
school_data = pd.read_csv(school_data_path)
student_data = pd.read_csv(student_data_path)

In [4]:
# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete_df = pd.DataFrame(school_data_complete)

In [5]:
# calculate the number of unique schools - using (nunique) in this case, on the complete df
unique_schools = school_data_complete_df["school_name"].nunique()

In [6]:
# Idea to use numpy sum  - From dan12345, https://stackoverflow.com/questions/20995196/pandas-counting-and-summing-specific-conditions
# lga is local government area, so the entire dataset

lga_reading = np.sum(school_data_complete_df["reading_score"] >=50)/school_data_complete_df["reading_score"].count()*100

In [7]:
lga_maths = np.sum(school_data_complete_df["maths_score"] >=50)/school_data_complete_df["maths_score"].count()*100

In [8]:
lga_overall = school_data_complete_df.query("maths_score >=50 and reading_score >=50")["reading_score"].count()/school_data_complete_df["reading_score"].count()*100

In [9]:
#Calculate the total number of students
total_students = school_data_complete_df["Student ID"].nunique()

In [10]:
# Total school budgets for LGA area. Drop duplicates info from: # https://www.w3schools.com/python/pandas/ref_df_drop_duplicates.asp#:~:text=The%20drop_duplicates()%20method%20removes,considered%20when%20looking%20for%20duplicates.
total_budget_2 = school_data_complete_df.drop_duplicates("budget")
all_schools_budget = total_budget_2["budget"].sum()


In [11]:
# Ave reading score
all_students_reading = school_data_complete_df["reading_score"].mean()

In [12]:
# Ave maths score
all_students_maths = school_data_complete_df["maths_score"].mean()

In [32]:
#Create a dataframe to hold the above results

data = [{"Total Schools":unique_schools, "Total Students":total_students, "Total Budget":all_schools_budget, "Average Maths Score":all_students_maths, "Average Reading Score":all_students_reading, "% Passing Maths":lga_maths, "% Passing Reading":lga_reading, "% Overall Passing":lga_overall}]
lga_summary = pd.DataFrame(data)
lga_summary

Total Schools  Total Students  Total Budget  Average Maths Score  \
0             15           39170      24649428            70.338192   

   Average Reading Score  % Passing Maths  % Passing Reading  \
0              69.980138        86.078632          84.426857   

   % Overall Passing  
0          72.808272

In [24]:
#Optional: give the displayed data cleaner formatting
#lga_summary = lga_summary.astype("Total Budget": float)
#lga_summary["Total Budget"] = lga_summary["Total Budget"].map("${:,.2f}".format)
#lga_summary["Total Students"] = lga_summary["Total Students"].apply('{:,}'.format)
#lga_summary.astype(float).map("${:,.2f}".format)

In [16]:
#Schools summary

In [35]:
#School reading and maths averages

average_df = school_data_complete_df[["school_name", "reading_score", "maths_score"]]
average_df.groupby(["school_name"]).mean()
school_data_means_df = pd.DataFrame(average_df.groupby(["school_name"]).mean())




In [26]:
school_data_by_school = pd.merge(school_data_means_df, school_data, how="left", on=["school_name", "school_name"])
school_data_by_school = school_data_by_school[['school_name','type','size','budget', 'maths_score' , 'reading_score']] 
school_data_by_school.rename(columns={'school_name':"School Name","type":"Type", 'size':"Total Students", "budget":"Budget",'maths_score':"Average Maths Score", 'reading_score':'Average Reading Score'}, inplace = True)
school_data_by_school
#school_data_by_school.set_index("School_Name", inplace=True)
PerStudent = school_data_by_school['Budget']/school_data_by_school['Total Students']
school_data_by_school.insert(3,"Per Student Budget",PerStudent,True)

In [27]:
school_data_by_school



School Name         Type  Total Students  Per Student Budget  \
0      Bailey High School   Government            4976               628.0   
1     Cabrera High School  Independent            1858               582.0   
2    Figueroa High School   Government            2949               639.0   
3        Ford High School   Government            2739               644.0   
4     Griffin High School  Independent            1468               625.0   
5   Hernandez High School   Government            4635               652.0   
6      Holden High School  Independent             427               581.0   
7       Huang High School   Government            2917               655.0   
8     Johnson High School   Government            4761               650.0   
9        Pena High School  Independent             962               609.0   
10  Rodriguez High School   Government            3999               637.0   
11    Shelton High School  Independent            1761               600.0   
12     Thomas High School  Independent            1635               638.0   
13     Wilson High School  Independent            2283               578.0   
14     Wright High School  Independent            1800               583.0   

     Budget  Average Maths Score  Average Reading Score  
0   3124928            72.352894              71.008842  
1   1081356            71.657158              71.359526  
2   1884411            68.698542              69.077993  
3   1763916            69.091274              69.572472  
4    917500            71.788147              71.245232  
5   3022020            68.874865              69.186408  
6    248087            72.583138              71.660422  
7   1910635            68.935207              68.910525  
8   3094650            68.843100              69.039277  
9    585858            72.088358              71.613306  
10  2547363            72.047762              70.935984  
11  1056600            72.034072              70.257808  
12  1043130            69.581651              69.768807  
13  1319574            69.170828              68.876916  
14  1049400            72.047222              70.969444

In [28]:
passread = []
passmaths = []
passboth = []
schoollist = school_data_by_school['School Name'].tolist()
for x in schoollist:
    newdf = school_data_complete_df[school_data_complete_df['school_name'] == x]
    passread.append(np.sum(newdf["reading_score"] >=50)/newdf["reading_score"].count()*100)
    passmaths.append(np.sum(newdf["maths_score"] >=50)/newdf["maths_score"].count()*100)
    passboth.append(newdf.query("maths_score >=50 and reading_score >=50")["reading_score"].count()/newdf["reading_score"].count()*100)

school_data_by_school.insert(7,"% Passing Reading",passread,True)
school_data_by_school.insert(7,"% Passing Maths",passmaths,True)
school_data_by_school.insert(9,"% Overall Passing",passboth,True)

school_data_by_school


School Name         Type  Total Students  Per Student Budget  \
0      Bailey High School   Government            4976               628.0   
1     Cabrera High School  Independent            1858               582.0   
2    Figueroa High School   Government            2949               639.0   
3        Ford High School   Government            2739               644.0   
4     Griffin High School  Independent            1468               625.0   
5   Hernandez High School   Government            4635               652.0   
6      Holden High School  Independent             427               581.0   
7       Huang High School   Government            2917               655.0   
8     Johnson High School   Government            4761               650.0   
9        Pena High School  Independent             962               609.0   
10  Rodriguez High School   Government            3999               637.0   
11    Shelton High School  Independent            1761               600.0   
12     Thomas High School  Independent            1635               638.0   
13     Wilson High School  Independent            2283               578.0   
14     Wright High School  Independent            1800               583.0   

     Budget  Average Maths Score  Average Reading Score  % Passing Maths  \
0   3124928            72.352894              71.008842        91.639871   
1   1081356            71.657158              71.359526        90.850377   
2   1884411            68.698542              69.077993        81.654798   
3   1763916            69.091274              69.572472        82.438846   
4    917500            71.788147              71.245232        91.212534   
5   3022020            68.874865              69.186408        80.949299   
6    248087            72.583138              71.660422        89.929742   
7   1910635            68.935207              68.910525        81.693521   
8   3094650            68.843100              69.039277        82.062592   
9    585858            72.088358              71.613306        91.683992   
10  2547363            72.047762              70.935984        90.797699   
11  1056600            72.034072              70.257808        91.538898   
12  1043130            69.581651              69.768807        83.853211   
13  1319574            69.170828              68.876916        82.785808   
14  1049400            72.047222              70.969444        91.777778   

    % Passing Reading  % Overall Passing  
0           87.379421          80.084405  
1           89.074273          80.785791  
2           82.807731          67.650051  
3           82.219788          67.469880  
4           88.487738          81.335150  
5           81.877023          66.364617  
6           88.524590          78.922717  
7           81.453548          66.712376  
8           81.978576          67.191766  
9           86.590437          79.209979  
10          87.396849          79.419855  
11          86.712095          78.875639  
12          82.629969          69.480122  
13          81.296540          67.455103  
14          86.666667          79.722222

In [30]:
#Top Performing Schools (By % Overall Passing)
#Sort and display the top five performing schools by % overall passing.
school_data_by_school.sort_values('% Overall Passing', ascending=False).head()


School Name         Type  Total Students  Per Student Budget  \
4     Griffin High School  Independent            1468               625.0   
1     Cabrera High School  Independent            1858               582.0   
0      Bailey High School   Government            4976               628.0   
14     Wright High School  Independent            1800               583.0   
10  Rodriguez High School   Government            3999               637.0   

     Budget  Average Maths Score  Average Reading Score  % Passing Maths  \
4    917500            71.788147              71.245232        91.212534   
1   1081356            71.657158              71.359526        90.850377   
0   3124928            72.352894              71.008842        91.639871   
14  1049400            72.047222              70.969444        91.777778   
10  2547363            72.047762              70.935984        90.797699   

    % Passing Reading  % Overall Passing  
4           88.487738          81.335150  
1           89.074273          80.785791  
0           87.379421          80.084405  
14          86.666667          79.722222  
10          87.396849          79.419855

In [31]:
#Bottom Performing Schools (By % Overall Passing)
#Sort and display the five worst-performing schools by % overall passing.
school_data_by_school.sort_values('% Overall Passing', ascending=True).head()

School Name         Type  Total Students  Per Student Budget  \
5   Hernandez High School   Government            4635               652.0   
7       Huang High School   Government            2917               655.0   
8     Johnson High School   Government            4761               650.0   
13     Wilson High School  Independent            2283               578.0   
3        Ford High School   Government            2739               644.0   

     Budget  Average Maths Score  Average Reading Score  % Passing Maths  \
5   3022020            68.874865              69.186408        80.949299   
7   1910635            68.935207              68.910525        81.693521   
8   3094650            68.843100              69.039277        82.062592   
13  1319574            69.170828              68.876916        82.785808   
3   1763916            69.091274              69.572472        82.438846   

    % Passing Reading  % Overall Passing  
5           81.877023          66.364617  
7           81.453548          66.712376  
8           81.978576          67.191766  
13          81.296540          67.455103  
3           82.219788          67.469880

In [24]:
student_data

Student ID     student_name gender  year         school_name  \
0               0     Paul Bradley      M     9   Huang High School   
1               1     Victor Smith      M    12   Huang High School   
2               2  Kevin Rodriguez      M    12   Huang High School   
3               3    Richard Scott      M    12   Huang High School   
4               4       Bonnie Ray      F     9   Huang High School   
...           ...              ...    ...   ...                 ...   
39165       39165     Donna Howard      F    12  Thomas High School   
39166       39166        Dawn Bell      F    10  Thomas High School   
39167       39167   Rebecca Tanner      F     9  Thomas High School   
39168       39168     Desiree Kidd      F    10  Thomas High School   
39169       39169  Carolyn Jackson      F    11  Thomas High School   

       reading_score  maths_score  
0                 96           94  
1                 90           43  
2                 41           76  
3                 89           86  
4                 87           69  
...              ...          ...  
39165             51           48  
39166             81           89  
39167             99           99  
39168             72           77  
39169             47           62  

[39170 rows x 7 columns]

In [49]:
#Create a table that lists the average maths score for students of each year level (9, 10, 11, 12) at each school.

#Create a pandas series for each year. Hint: use a conditional statement.

#Group each series by school

#Combine the series into a dataframe

#Optional: give the displayed data cleaner formatting Maths Scores by Year


only_nine = student_data.loc[student_data["year"] == 9 ]
only_ten = student_data.loc[student_data["year"] == 10]
only_eleven = student_data.loc[student_data["year"] == 11]
only_twelve = student_data.loc[student_data["year"] == 12]


nine_m_ave = only_nine.groupby(['school_name']).mean()["maths_score"]
ten_m_ave = only_ten.groupby(['school_name']).mean()["maths_score"]
eleven_m_ave = only_eleven.groupby(['school_name']).mean()["maths_score"]
twelve_m_ave = only_twelve.groupby(['school_name']).mean()["maths_score"]


Maths_scores_by_level = pd.DataFrame([nine_m_ave, ten_m_ave, eleven_m_ave, twelve_m_ave])
Maths_scores_by_level = Maths_scores_by_level.T
Maths_scores_by_level.columns = ['Year 9', 'Year 10', 'Year 11', 'Year 12']
                                      
Maths_scores_by_level






C:\Users\rhian\AppData\Local\Temp\ipykernel_16572\805126290.py:18: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  nine_ave = only_nine.groupby(['school_name']).mean()["maths_score"]
C:\Users\rhian\AppData\Local\Temp\ipykernel_16572\805126290.py:19: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ten_ave = only_ten.groupby(['school_name']).mean()["maths_score"]
C:\Users\rhian\AppData\Local\Temp\ipykernel_16572\805126290.py:20: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select onl

Year 9    Year 10    Year 11    Year 12
school_name                                                      
Bailey High School     72.493827  71.897498  72.374900  72.675097
Cabrera High School    72.321970  72.437768  71.008299  70.604712
Figueroa High School   68.477804  68.331586  68.811001  69.325282
Ford High School       69.021609  69.387006  69.248862  68.617811
Griffin High School    72.789731  71.093596  71.692521  71.469178
Hernandez High School  68.586831  68.867156  69.154412  68.985075
Holden High School     70.543307  75.105263  71.640777  73.409639
Huang High School      69.081754  68.533246  69.431345  68.639316
Johnson High School    69.469286  67.990220  68.637730  69.287393
Pena High School       71.996364  72.396000  72.523438  71.187845
Rodriguez High School  71.940722  71.779808  72.364811  72.154626
Shelton High School    72.932075  72.506696  70.097087  72.331536
Thomas High School     69.234273  70.057007  69.657831  69.369822
Wilson High School     69.212361  69.455446  68.378965  69.787472
Wright High School     71.741176  72.179012  73.275862  70.848238

In [34]:
average_df = school_data_complete_df[["school_name", "year", "maths_score"]]

#school_data_means_df = pd.DataFrame(average_df.groupby(["school_name","year"]).mean("maths_score"))


ser

#school_data_means_df



ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
# From Tom Piesel:
# def aggregate_schools(d):
#     school_type = d.type.iloc[0]
#     total_students = d.student_name.count()
#     avg_maths = d.maths_score.mean()
#     # etc 

#     labels = [
#         'School Type',
#         'Total Students',
#         'Average Maths Score'
#         # etc
   
#     return d.Series([
#         school_type,
#         total_students,
#         avg_maths
#         # etc
#     ], index=labels)

# merged_dataframe.groupby('school_name').apply(aggregate_schools)

In [ ]:
# Create a pandas series for each year. Hint: use a conditional statement. 

# Group each series by school

# Combine the series into a dataframe

# Optional: give the displayed data cleaner formatting

In [52]:
# Reading Score by Year
# Perform the same operations as above for reading scores

nine_r_ave = only_nine.groupby(['school_name']).mean()["reading_score"]
ten_r_ave = only_ten.groupby(['school_name']).mean()["reading_score"]
eleven_r_ave = only_eleven.groupby(['school_name']).mean()["reading_score"]
twelve_r_ave = only_twelve.groupby(['school_name']).mean()["reading_score"]


Read_scores_by_level = pd.DataFrame([nine_r_ave, ten_r_ave, eleven_r_ave, twelve_r_ave])
Read_scores_by_level = Read_scores_by_level.T
Read_scores_by_level.columns = ['Year 9', 'Year 10', 'Year 11', 'Year 12']
                                      
Read_scores_by_level


C:\Users\rhian\AppData\Local\Temp\ipykernel_16572\1286665914.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  nine_r_ave = only_nine.groupby(['school_name']).mean()["reading_score"]
C:\Users\rhian\AppData\Local\Temp\ipykernel_16572\1286665914.py:5: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ten_r_ave = only_ten.groupby(['school_name']).mean()["reading_score"]
C:\Users\rhian\AppData\Local\Temp\ipykernel_16572\1286665914.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or se

Year 9    Year 10    Year 11    Year 12
school_name                                                      
Bailey High School     70.901920  70.848265  70.317346  72.195525
Cabrera High School    71.172348  71.328326  71.201245  71.856021
Figueroa High School   70.261682  67.677588  69.152327  69.082126
Ford High School       69.615846  68.988701  70.735964  68.849722
Griffin High School    72.026895  70.746305  72.385042  69.434932
Hernandez High School  68.477569  70.621842  68.418199  69.244136
Holden High School     71.598425  71.096491  73.310680  70.481928
Huang High School      68.670616  69.516297  68.740638  68.671795
Johnson High School    68.719286  69.295029  69.969115  67.992521
Pena High School       70.949091  72.324000  71.703125  71.513812
Rodriguez High School  70.902921  70.137500  71.424453  71.414449
Shelton High School    70.715094  69.879464  71.150485  69.070081
Thomas High School     69.672451  69.741093  70.749398  68.730769
Wilson High School     68.683043  68.412541  68.796327  69.888143
Wright High School     71.823529  71.386831  70.475862  69.821138

In [ ]:
# Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable
# bins to group school spending. Include in the table each of the following:
#Average Maths Score
#Average Reading Score
#% Passing Maths
#% Passing Reading
#Overall Passing Rate (Average of the above two)

# Create a table that breaks down school performances based on average Spending Ranges (Per Student). 

# USE THE FOLLOWING CODE FOR THE BINNING

spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

df["Name"] = pd.cut(Name["ColumnName"], spending_bins, labels=labeld, include_lowest=True)
df

#pd.cut to categorise spending based on the bins.
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Math Score"].mean()
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Math"].mean()
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Reading"].mean()
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Overall Passing"].mean()

#Use the scores above to create a DataFrame called spending_summary.

#Include the following metrics in the table:
# Average Maths Score
# Average Reading Score
# % Passing Maths
# % Passing Reading
# Overall Passing Rate (Average of the above two)

#	Average Maths Score	Average Reading Score	% Passing Maths	% Passing Reading	% Overall Passing
#Spending Ranges (Per Student)					
# <$585	71.36	70.72	88.84	86.39	76.72
# $585-630	72.07	71.03	91.52	87.29	79.88
# $630-645	69.85	69.84	84.69	83.76	71.00
# $645-680	68.88	69.05	81.57	81.77	66.76


In [ ]:
# Scores by School Size
# Perform the same operations as above, based on school size.

In [ ]:
# Scores by School Type
# Perform the same operations as above, based on school type